# Прикладные задачи анализа данных

## Домашнее задание 4

В этой работе мы будем решать задачу рекомендаций музыки. Для этого мы реализуем метод коллаборативной фильтрации и модель со скрытыми переменными, сравним их между собой и попытаемся сделать выводы. Во всех заданиях разрешается переписывать код по своему усмотрению, предложенные шаблоны несут лишь рекомендательный характер. За дополнительные эксперименты с методами и их анализ можно получить бонусные баллы.

In [ ]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Callable, List

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as scs
import scipy as sc

import warnings

In [ ]:
ratings = pd.read_csv('music_dataset.csv')
ratings.head()

In [ ]:
tracks_info = pd.read_csv('tracks_info.csv')
tracks_info.head()

Для оценки качества рекомендаций мы будем использовать метрику $MAP@k$.

$$
MAP@k = \frac{1}{N} \sum_{u = 1}^N AP_u@k
$$
$$
AP_u@k = \frac{1}{\min(k, n_u)} \sum_{i=1}^k r_u(i) p_u@i
$$
$$p_u@k = \dfrac{1}{k}\sum_{j=1}^k r_u(j)$$


*   $N$ - количество пользователей.
*   $n_u$ - число релевантных треков пользователя $u$ на тестовом промежутке.
*   $r_u(i)$ - бинарная величина: относится ли трек на позиции $i$ к релевантным.

**Задание 1 (0.5 балла).** Реализуйте метрику $MAP@k$.

In [ ]:
def pk(r):
    return sum(r[:k])/k

def mapk(relevant: List[List[int]], predicted: List[List[int]], k: int = 20):
    map_k = 0
    for i,pred in enumerate(predicted):
#         print(i)
        r_u = np.in1d(pred,relevant[i])
        ap_k = 0
        for j in range(k):
            ap_k += r_u[j]*sum(r_u[:j+1])/(j+1)
        ap_k *= 1/min(k,len(relevant[i])) 
        map_k += ap_k
    return map_k/(i+1)

In [ ]:
relevant = [
    [1, 7, 6, 2, 8],
    [1, 5, 4, 8],
    [8, 2, 5]
]

pred = [
    [8, 1, 5, 0, 7, 2, 9, 4],
    [0, 1, 8, 5, 3, 4, 7, 9],
    [9, 2, 0, 6, 8, 5, 3, 7]
]
print(round(mapk(relevant, pred, k=5), 4))
assert round(mapk(relevant, pred, k=5), 4) == 0.4331

Разделим данные на тренировочные и тестовые так, чтобы в теcтовый датасет попали 50 последних треков каждого пользователя.

In [ ]:
def train_test_split(ratings):
    train_ratings, test_ratings = [], []
    num_test_samples = 50

    # getting train samples
    for userId, user_data in tqdm(ratings.groupby('userId')):
        train_ratings += [user_data[:-num_test_samples]]

    train_ratings = pd.concat(train_ratings).reset_index(drop=True)
    all_train_items = train_ratings['trackId'].unique()

    # getting train samples
    # we drop all tracks that are not presented it the training samples,
    # because we won't be able to learn representations for them
    for userId, user_data in tqdm(ratings.groupby('userId')):
        test_items = user_data[-num_test_samples:]
        test_items = test_items[np.isin(test_items['trackId'], all_train_items)]
        test_ratings += [test_items]

    test_ratings = pd.concat(test_ratings).reset_index(drop=True)

    return train_ratings, test_ratings

In [ ]:
train_ratings, test_ratings = train_test_split(ratings)

Почистим табличку с информацией о треках и закодируем id треков так, чтобы они соответствовали их порядковому номеру.

In [ ]:
redundant_rows = np.where(~np.isin(tracks_info['id'], train_ratings['trackId'].unique()))[0]
tracks_info.drop(redundant_rows, inplace=True)
tracks_info = tracks_info.reset_index(drop=True)

In [ ]:
def ids_encoder(ratings):
    users = sorted(ratings['userId'].unique())
    items = sorted(ratings['trackId'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    return uencoder, iencoder

In [ ]:
uencoder, iencoder = ids_encoder(train_ratings)
train_ratings['trackId'] = iencoder.transform(train_ratings['trackId'].tolist())
test_ratings['trackId'] = iencoder.transform(test_ratings['trackId'].tolist())
tracks_info['id'] = iencoder.transform(tracks_info['id'].tolist())

In [ ]:
train_ratings.head()

In [ ]:
test_ratings.head()

Соберем все релевантные треки для каждого пользователя в список.

In [ ]:
test_relevant = []
test_users = []
for user_id, user_data in test_ratings.groupby('userId'):
    test_relevant += [user_data['trackId'].tolist()]
    test_users.append(user_id)

In [ ]:
train_relevant = []
train_users = []
for user_id, user_data in train_ratings.groupby('userId'):
    train_relevant += [user_data['trackId'].tolist()]
    train_users.append(user_id)

**Задание 2 (0.5 балла).** Реализуйте метод `get_test_recommendations` в классе `BaseModel`. Он принимает на вход параметр `k` и возвращает массив из `k` наиболее подходящих треков для каждого пользователя. Не забывайте удалять уже прослушанные треки из рекомендуемых.

In [ ]:
class BaseModel:
    def __init__(self, ratings: pd.DataFrame):
        self.ratings = ratings
        self.n_users = len(np.unique(self.ratings['userId']))
        self.n_items = len(np.unique(self.ratings['trackId']))

        self.R = np.zeros((self.n_users, self.n_items))
        self.R[self.ratings['userId'], self.ratings['trackId']] = 1.
#         print(self.R)
        
    def recommend(self, uid: int):
        """
        param uid: int - user's id
        return: [n_items] - vector of recommended items sorted by their scores in descending order
        """
        raise NotImplementedError

    def remove_train_items(self, preds: List[List[int]], k: int):
        """
        param preds: [n_users, n_items] - recommended items for each user
        param k: int
        return: np.array [n_users, k] - recommended items without training examples
        """
        new_preds = np.zeros((len(preds), k), dtype=int)
        for user_id, user_data in self.ratings.groupby('userId'):
            user_preds = preds[user_id]
            new_preds[user_id] = user_preds[~np.in1d(user_preds, user_data['trackId'])][:k]
        return new_preds

    def get_test_recommendations(self, k: int,test_users:List[int]):
        test_preds = []
        for uid in np.unique(self.ratings['userId']):
            test_preds.append(self.recommend(uid))
#             print(test_preds[-1])
        test_preds = self.remove_train_items(test_preds,k)
        return test_preds[test_users]

## Часть 1. Коллаборативная фильтрация (User2User)

Идея: чтобы выбрать треки, которые понравятся пользователю, можно набрать несколько похожих на него пользователей (соседей) и посмотреть, какие треки они слушают. После этого остается агрегировать треки этих пользователей и выбрать самые популярные. Соответственно, задача состоит из двух частей: выбора функции похожести двух пользователей и способа агрегации.

В качестве функции похожести мы будем использовать две метрики:

1. Корреляция Пирсона $$s(u, v) = \frac{\sum_{i \in I_u \cap I_v} r_{ui}r_{vi}}{\sqrt{\sum_{i \in I_u} r_{ui} ^2}\sqrt{\sum_{i \in I_v} r_{vi}^2}} $$

2. Мера Жаккара

$$ s(u, v) = \frac{|I_u \cap I_v|}{|I_u \cup I_v|} $$


Корреляция Пирсона немного видоизменена, чтобы подходить под нашу задачу.


Во всех формулах 
* $I_u$ - множество треков, прослушанных пользователем $u$.
* $r_{ui}$ - прослушал ли пользователь $u$ трек $i$ (0 или 1).

Множество соседей определим как $$N(u) = \{ v \in U \setminus \{u\} \mid s(u, v) > \alpha\},$$ где $\alpha \, - $ гиперпараметр.



Для агрегации мы будем пользоваться следующей формулой.
$$
\hat{r}_{ui} = \frac{\sum_{v \in N(u)} s(u, v) r_{vi}}{\sum_{v \in N(u)} |s(u, v)|}
$$

**Задание 3.1 (0.5 балла).** Реализуйте функцию подсчета корреляции Пирсона.

**Задание 3.2 (0.5 балла).** Реализуйте функцию подсчета меры Жаккара.

Функции принимают матрицу оценок и вектор оценок пользователя $u$ и возвращают вектор со значениями похожести пользователя $u$ на всех пользователей. Старайтесь писать оптимизированный код, за неэффективную реализацию оценка может быть снижена.

In [ ]:
def pearson(ratings: np.array, user_vector: np.array) -> np.array:
    s = np.divide(ratings@user_vector.T,np.sqrt(np.sum(ratings,1))*np.sqrt(np.sum(user_vector)))
    mask = s ==1
    s[mask] = -1
    return s


def jaccard(ratings: np.array, user_vector: np.array) -> np.array:
    s = np.divide(ratings@user_vector.T,np.sum(ratings+user_vector!=0,1))
    mask = s ==1
    s[mask] = -1
    return s


def r_and(ratings: np.array, user_vector: np.array) -> np.array:
    return np.random.rand(ratings.shape[0])  

**Задание 4 (1 балл).** Реализуйте методы `similarity` и `recommend` класса `User2User`. `recommend` возвращает индексы треков, отсортированные в порядке убывания предсказанных оценок.


In [ ]:
class User2User(BaseModel):
    def __init__(self, ratings, similarity_func,sparse: bool):
        super().__init__(ratings)

        assert similarity_func in [pearson, jaccard,r_and]
        self.sparse = sparse
        self.similarity_func = similarity_func
        self.alpha = 0.02
        if self.sparse:
            self.R = scs.csr_matrix(self.R)

    def similarity(self, user_vector: np.array):
        """
        user_vector: [n_items]
        """
        if self.sparse:
            user_vector = user_vector.toarray()
        s = np.asarray(self.similarity_func(self.R,user_vector)).squeeze()
        mask = s>self.alpha
        s = s[mask]
#         print(s)
        new_r = s@self.R[mask]/np.sum(s)
        return new_r
        

    def recommend(self, uid: int):
        r = self.similarity(self.R[uid])
#         print(r)
#         print(np.argsort(r))
        return np.argsort(r)[::-1]

**Задание 5 (1 балл).** Постройте графики зависимости значений $MAP@k$ от разных $k$ для обоих функций похожести, сравните их между собой, а также с предсказаниями случайного алгоритма и сделайте выводы.

In [ ]:
warnings.filterwarnings("default")

In [ ]:
model_j = User2User(train_ratings, jaccard,False)
model_p = User2User(train_ratings, pearson,False)
model_r = User2User(train_ratings, r_and,False)
# print(model_j.remove_train_items(test_relevant,5,test_users))
map_k_j=[]
map_k_p=[]
map_k_r=[]
# print(train_relevant[0])
for k in tqdm(range(1,51)):
#     print(model_p.get_test_recommendations(k,test_users))
#     print(test_relevant)
    map_k_j.append(mapk(test_relevant,model_j.get_test_recommendations(k,test_users),k))
    map_k_p.append(mapk(test_relevant,model_p.get_test_recommendations(k,test_users),k))
    map_k_r.append(mapk(test_relevant,model_r.get_test_recommendations(k,test_users),k))
f,axes = plt.subplots(1,2)
f1,axes1 = plt.subplots(1)
axes[0].set_title("jaccard")
axes[1].set_title("pearson")
axes1.set_title("random")
axes[0].plot(map_k_j,color='orange')
axes[1].plot(map_k_p,color='green')
axes1.plot(map_k_r)

**Задание 6 (1.5 балла).** Как вы могли заметить, матрица оценок получается очень разреженной, но мы работаем с ней как с обычной, это не дело. Перепишите код так, чтобы все методы могли работать с разреженными матрицами и сравните скорость работы такого подхода с оригинальным.

In [ ]:
model_j = User2User(train_ratings, jaccard,True)
model_p = User2User(train_ratings, pearson,True)
model_r = User2User(train_ratings, r_and,True)
map_k_j=[]
map_k_p=[]
map_k_r=[]
for k in tqdm(range(1,51)):
    map_k_j.append(mapk(test_relevant,model_j.get_test_recommendations(k,test_users),k))
    map_k_p.append(mapk(test_relevant,model_p.get_test_recommendations(k,test_users),k))
    map_k_r.append(mapk(test_relevant,model_r.get_test_recommendations(k,test_users),k))
f,axes = plt.subplots(1,2)
f1,axes1 = plt.subplots(1)
axes[0].set_title("jaccard")
axes[1].set_title("pearson")
axes1.set_title("random")
axes[0].plot(map_k_j,color='orange')
axes[1].plot(map_k_p,color='green')
axes1.plot(map_k_r)

Мы можем посмотреть глазами, насколько хорошо модель рекомендует треки. Для этого сравним уже прослушанные треки с рекомендованными и релевантными для случайного пользователя. Хорошо ли у вас получилось?

In [ ]:
user_id = np.random.randint(0, model_j.n_users)

In [ ]:
listened_tracks = train_ratings[train_ratings.userId == user_id].trackId[:15]

print('Already listened tracks:')

tracks_info.loc[listened_tracks][['name', 'artists']]

In [ ]:
preds = np.asarray(model_j.get_test_recommendations(15,[user_id])).squeeze()

print('Predicted tracks:')

tracks_info.loc[preds][['name', 'artists']]

In [ ]:
test_tracks = test_ratings[test_ratings.userId == user_id].trackId[:15]

print('Test-time tracks:')

tracks_info.loc[test_tracks][['name', 'artists']]

### Часть 2. Модель со скрытыми переменными

В этой части мы пощупаем метод рекомендаций со скрытыми переменными.
Идея: будем предсказывать оценки по формуле
$$
\hat{r}_{ui} = \langle p_u, q_i \rangle,
$$
$p_u \in \mathbb{R}^d$ и $q_i \in \mathbb{R}^d$ - латентные векторы пользователя $u$ и объекта $i$ соответственно. 

Оптимизировать мы будем MSE между истинной оценкой пользователя и предсказанной с регуляризацией
$$
L = \sum_{(u, i) \in R} (\hat{r}_{ui} - r_{ui})^2 + \lambda \left(\sum_{u \in U} \|p_u\|^2 + \sum_{i \in I} \|q_i\|^2\right)
$$

**Задание 7 (1.5 балла).** На лекции рассматривались два подхода к оптимизации параметров. Можно это делать обычным стохастческим градинтным спуском, а можно по очереди обновлять матрицы $P, Q$, и тогда получится метод Alternating Least Squares (ALS). Выведите формулы обновления параметров для обоих методов.

**LFM:**

Ответ

На каждом шаге выбираем пару $(u,i) \in R$.
$$\vec{p_u} = \vec{p_u}\left(1-\eta\lambda\right) - \vec{q_i}\eta\left(\langle\vec{p_u},\vec{q_i}\rangle-r_{ui}\right)$$ 
$$\vec{q_i} = \vec{q_i}\left(1-\eta\lambda\right) - \vec{p_u}\eta\left(\langle\vec{p_u},\vec{q_i}\rangle-r_{ui}\right)$$ 
$\eta$ - шаг спуска.



**ALS:**

Ответ

Для $P$:
$$P = \left(QQ^T+\lambda E_d\right)^{-1}QR^T $$

Для $Q$:
$$Q = \left(PP^T+\lambda E_d\right)^{-1}PR $$

Здесь $E_d$ -единичная матрица размером $d\times d$.


**Задание 8 (2 балла).** Реализуйте методы оптимизации параметров для обоих алгоритмов.

In [ ]:
class HiddenVars(BaseModel):
    def __init__(self, ratings, dim=128, mode='sgd',lamb=0.01):
        super().__init__(ratings)
        self.dim = dim
        
        assert mode in ['sgd', 'als']
        self.mode = mode

#         self.P = np.random.normal(size=(self.n_users, dim))
#         self.Q = np.random.normal(size=(self.n_items, dim))
        
        self.P = (np.ones((self.n_users, dim))+0.1*np.random.normal(size=(self.n_users, dim)))/(dim**(0.5)*2)
        self.Q = (np.ones((self.n_items, dim))+0.1*np.random.normal(size=(self.n_items, dim)))/(dim**(0.5)*2)

        self.lr = 0.0003
        self.lamb = lamb
        self.R = scs.csr_matrix(self.R)
        self.flag = 0

    def fit(self, num_iters=5):
        loss = 0
#         
        for epoch in tqdm(range(num_iters)):
        
            if self.mode == 'sgd':
                ind = np.random.randint(len(self.ratings))
                ind = self.ratings.iloc[ind].to_numpy()
#                 print(ind)
                p,q = self.P[ind[0]],self.Q[ind[1]]
#                 print(p)
                self.P[ind[0]] = p*(1-self.lr*self.lamb) - q*self.lr*(p*q.T-1)
#                 print(self.P[ind[0]])
                self.Q[ind[1]] = q*(1-self.lr*self.lamb) - p*self.lr*(p*q.T-1) 
#                 self.P[ind[0]] = p*(1-self.lr*self.lamb) - q*self.lr*(p*q.T-1)
#                 self.Q[ind[1]] = q*(1-self.lr*self.lamb) - p*self.lr*(p*p.T-1) 

            elif self.mode == 'als':
                A = self.Q.T@self.Q + self.lamb*np.eye(self.dim)
                self.P = scs.csr_matrix(self.R)@self.Q@np.linalg.inv(A)
                A = self.P.T@self.P + self.lamb*np.eye(self.dim)
                self.Q = scs.csr_matrix(self.R).T@self.P@np.linalg.inv(A)
            if epoch==1000000:
#             self.lamb=1
                self.lr = self.lr/10
        loss = sc.linalg.norm(self.R- self.P@self.Q.T)**2+self.lamb*(np.sum(self.P**2)+np.sum(self.Q**2))
        if self.flag==20:
#             self.lamb=1
            self.lr = self.lr/10
            print(self.lr)
#             self.flag=0
        return loss
            
                

    def recommend(self, uid):
        pred_rating = self.P[uid] @ self.Q.T

        return np.argsort(pred_rating)[::-1]

**Задание 9 (1 балл).** Для обоих алгоритмов подберите оптимальные значения размерности скрытого пространства $d$ и размера предсказания $k$. Как изменяется качество предсказаний с числом итераций обучения? Постройте соответствующие графики, сравните со случайным подхом и User2User, сделайте выводы. Какой алгоритм вам кажется более подходящим для данной задачи и почему?

P.S. Хотя бы один из методов обучения должен приводить к лучшим результатам в сравнении с User2User подходом.

P.P.S. Методу LFM свойственно переобучаться, поэтому при подборе параметров полезно смотреть на значения ошибки и оптимизируемой метрики на тренировочном датасете. Вы также можете менять начальную инициализацию и прочие параметры, за исключением архитектуры, на ваш вкус.

In [ ]:
warnings.filterwarnings("default")

In [ ]:
print(len(train_ratings))

In [ ]:
N=3*100000

In [ ]:
map_k_sgd_tr=[]
map_k_sgd_te=[]
num_iters = []
loss = []
# for dim in 32,64,128]
dim=128
model_sgd = HiddenVars(train_ratings,dim=dim)
for i in range(50):
    loss.append(model_sgd.fit((N)))
    map_k_sgd_te.append(mapk(test_relevant,model_sgd.get_test_recommendations(5,test_users),5))
    map_k_sgd_tr.append(mapk(train_relevant,model_sgd.get_test_recommendations(5,train_users),5))
    num_iters.append(i+1)
    f,axes = plt.subplots(1,2,figsize=(26, 14))
    axes[0].grid()
    axes[1].grid()
    f.suptitle("SGD, dim="+str(dim))
    axes[1].set_title("loss 10 last elements")
    axes[0].set_title("map_k test")
    axes[1].plot(num_iters[-10:],loss[-10:],color='orange')
    axes[0].plot(num_iters,map_k_sgd_te,color='green')
    f1,axes1 = plt.subplots(1,figsize=(26, 14))
    axes1.set_title("loss")
    axes1.plot(num_iters,loss)
    axes1.grid()
    plt.show()


In [ ]:
map_k_als_tr=[]
map_k_als_te=[]
num_iters = []
loss_als = []
model_als = HiddenVars(train_ratings,dim=dim,mode='als',lamb=1)
for i in range(20):
    loss_als.append(model_als.fit(1))
    map_k_als_te.append(mapk(test_relevant,model_als.get_test_recommendations(5,test_users),5))
#     map_k_als_tr.append(mapk(train_relevant,model_als.get_test_recommendations(5,train_users),5))
    num_iters.append(i+1)
    f,axes = plt.subplots(1,2,figsize=(26, 14))
    axes[0].grid()
    axes[1].grid()
    f.suptitle("SGD, dim="+str(dim))
    axes[1].set_title("loss 5 items")
    axes[0].set_title("map_k test")
    axes[0].plot(num_iters,map_k_als_te,color='orange')
    axes[1].plot(num_iters[-5:],loss_als[-5:],color='green')
    f1,axes1 = plt.subplots(1,figsize=(26, 14))
    axes1.set_title("loss")
    axes1.plot(num_iters,loss_als)
    axes1.grid()
    plt.show()


In [ ]:
for dim in [32,64,128,256,512,1024]:
    model_als = HiddenVars(train_ratings,dim=dim,mode='als',lamb=0.01)
    for k in range(1,15):
        map_k_als_tr=[]
        map_k_als_te=[]
        num_iters = []
        loss_als = []
        for i in range(4):
            loss_als.append(model_als.fit(1))
            map_k_als_te.append(mapk(test_relevant,model_als.get_test_recommendations(k,test_users),k))
            #     map_k_als_tr.append(mapk(train_relevant,model_als.get_test_recommendations(5,train_users),5))
            num_iters.append(i+1)
            f,axes = plt.subplots(1,2,figsize=(26, 14))
            axes[0].grid()
            axes[1].grid()
            f.suptitle("ALS, dim="+str(dim)+"k="+str(k))
            axes[1].set_title("loss 5 items")
            axes[0].set_title("map_k test")
            axes[0].plot(num_iters,map_k_als_te,color='orange')
            axes[1].plot(num_iters[-5:],loss_als[-5:],color='green')
            f1,axes1 = plt.subplots(1,figsize=(26, 14))
            axes1.set_title("loss")
            axes1.plot(num_iters,loss_als)
            axes1.grid()
            plt.show()


In [ ]:
for dim in [512,1024]:
    model_als = HiddenVars(train_ratings,dim=dim,mode='als',lamb=0.01)
    for k in range(1,15):
        map_k_als_tr=[]
        map_k_als_te=[]
        num_iters = []
        loss_als = []
        for i in range(10):
            loss_als.append(model_als.fit(1))
            map_k_als_te.append(mapk(test_relevant,model_als.get_test_recommendations(k,test_users),k))
            #     map_k_als_tr.append(mapk(train_relevant,model_als.get_test_recommendations(5,train_users),5))
            num_iters.append(i+1)
            f,axes = plt.subplots(1,2,figsize=(26, 14))
            axes[0].grid()
            axes[1].grid()
            f.suptitle("ALS, dim="+str(dim)+"k="+str(k))
            axes[1].set_title("loss 5 items")
            axes[0].set_title("map_k test")
            axes[0].plot(num_iters,map_k_als_te,color='orange')
            axes[1].plot(num_iters[-5:],loss_als[-5:],color='green')
            f1,axes1 = plt.subplots(1,figsize=(26, 14))
            axes1.set_title("loss")
            axes1.plot(num_iters,loss_als)
            axes1.grid()
            plt.show()

In [ ]:
model_als = HiddenVars(train_ratings,dim=128,mode='als',lamb=0.01)
# map_k_als_tr=[]
# map_k_als_te=[]
# num_iters = []
# loss_als = []
model_als.fit(3)
# loss_als.append(model_als.fit(3))
print("MAP@k=",mapk(test_relevant,model_als.get_test_recommendations(1,test_users),1))
#     map_k_als_tr.append(mapk(train_relevant,model_als.get_test_recommendations(5,train_users),5))
num_iters.append(i+1)
# f,axes = plt.subplots(1,2,figsize=(26, 14))
# axes[0].grid()
# axes[1].grid()
# f.suptitle("ALS, dim="+str(dim)+"k="+str(k))
# axes[1].set_title("loss 5 items")
# axes[0].set_title("map_k test")
# axes[0].plot(num_iters,map_k_als_te,color='orange')
# axes[1].plot(num_iters[-5:],loss_als[-5:],color='green')
# f1,axes1 = plt.subplots(1,figsize=(26, 14))
# axes1.set_title("loss")
# axes1.plot(num_iters,loss_als)
# axes1.grid()
# plt.show()

По LFM модели: вообще не обучается, долго запускал. Если ставить вес регурялизации меньше единицы, лосс вообще растет. Если ставить меньше то оубчение просто ужасно проходит. Долго и без результата.

По ALS модели: обучается легко. Лучшие параметры ($d$=128, $k$=1, $epoches$=3)  Хорошое качество лучше чем модель USer2User или рандомная. Также я пытался использовать последовательно ALS инициализацию матриц Q и P, и дальнейшее обучение модели LFM, однако все также плохо по качеству.

Если у вас получилось достаточно хорошее качество, то при оптимизации параметров марицы $Q$ похожим трекам стали соответствовать похожие векторы. Поэтому мы можем для любого трека найти наиболее близкие к нему в латентном пространстве и проверить степерь обученности модели вручную.

In [ ]:
example_trackId = tracks_info[tracks_info.name == 'Выхода нет'].iloc[0].id

preds = model_als.Q @ model_als.Q[example_trackId]
preds = preds / np.sqrt((model_als.Q**2).sum(axis=1) + 1e-8)

track_idxs = preds.argsort()[::-1][:20]

In [ ]:
similar_tracks = tracks_info.loc[track_idxs][['name', 'artists']]
similar_tracks['similarity'] = preds[track_idxs] / np.linalg.norm(model_als.Q[example_trackId])
similar_tracks